In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow.keras import layers
import time
from tensorflow import data
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense,ZeroPadding2D, BatchNormalization, Activation, Layer, ReLU, LeakyReLU,Conv2D,AveragePooling2D,UpSampling2D,Reshape,Flatten
from tensorflow.keras.models import Model


In [2]:
BATCH_SIZE = 1024


In [3]:
data = np.load('tcPred.npy')

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(60000).batch(1024)


Metal device set to: Apple M2 Max


2023-02-24 14:57:10.775332: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-24 14:57:10.775355: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
data.shape

(21263, 147)

In [6]:
DSHAPE = 147

In [7]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))

    model.add(layers.Dense(DSHAPE, activation='relu', use_bias=False, input_shape=(DSHAPE,)))


    

    return model

In [8]:

noise = tf.random.normal([1, DSHAPE])


In [9]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))
   
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [10]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()


In [11]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

In [12]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [13]:
generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

In [17]:
checkpoint_dir = './training_checkpoints3'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [18]:
EPOCHS = 40
noise_dim = DSHAPE
num_examples_to_generate = 50

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [19]:
train_dataset

<BatchDataset element_spec=TensorSpec(shape=(None, 147), dtype=tf.float32, name=None)>

In [20]:
def train_step(images, gen_losses, disc_losses):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)

        disc_loss = discriminator_loss(real_output, fake_output)
        gen_losses = gen_losses.append(gen_loss.numpy())
        disc_losses = disc_losses.append(disc_loss.numpy())
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [21]:

def train(dataset, epochs, gen_losses, disc_losses, gloss, dloss):
  for epoch in range(epochs):
    start = time.time()


    for image_batch in dataset:
        train_step(image_batch, gen_losses,disc_losses)
    print("gen_loss =" + str(gen_losses[-1]))
    print("disc_loss =" + str(disc_losses[-1]))
    gloss.append(gen_losses[-1])
    dloss.append(disc_losses[-1])

    # Produce images for the GIF as you go
    x = generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  saved = generate_and_save_images(generator,
                           epochs,
                           seed)
  return saved

In [22]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)
  return predictions


In [23]:
gen_losses = []
disc_losses = []
gloss = []
dloss = []

In [24]:
discriminator = make_discriminator_model()
generator = make_generator_model()

In [25]:
final = train(train_dataset,40, gen_losses, disc_losses, gloss, dloss)


gen_loss =0.7598083
disc_loss =1.3518896
Time for epoch 1 is 1.876565933227539 sec
gen_loss =0.82013905
disc_loss =1.2123873
Time for epoch 2 is 1.207535982131958 sec
gen_loss =0.9826574
disc_loss =0.9979788
Time for epoch 3 is 1.2297310829162598 sec
gen_loss =1.2618387
disc_loss =0.76745796
Time for epoch 4 is 1.2117040157318115 sec
gen_loss =1.581557
disc_loss =0.60295224
Time for epoch 5 is 1.1917388439178467 sec
gen_loss =1.939887
disc_loss =0.4535389
Time for epoch 6 is 1.1821820735931396 sec
gen_loss =2.2210493
disc_loss =0.38744238
Time for epoch 7 is 1.1916718482971191 sec
gen_loss =2.448646
disc_loss =0.3535519
Time for epoch 8 is 1.210179090499878 sec
gen_loss =2.6446161
disc_loss =0.31141812
Time for epoch 9 is 1.2960278987884521 sec
gen_loss =2.7952325
disc_loss =0.2839938
Time for epoch 10 is 1.2748379707336426 sec
gen_loss =2.8918185
disc_loss =0.3407809
Time for epoch 11 is 1.2347829341888428 sec
gen_loss =2.9161575
disc_loss =0.31672424
Time for epoch 12 is 1.2121498584

In [26]:
final.eval

<bound method _EagerTensorBase.eval of <tf.Tensor: shape=(50, 147), dtype=float32, numpy=
array([[0.41432533, 0.65544325, 0.28528917, ..., 0.        , 0.        ,
        0.        ],
       [0.3588624 , 0.42854884, 0.41608822, ..., 0.        , 0.        ,
        0.        ],
       [0.53015906, 0.08060289, 0.5188161 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.12347867, 0.        , 0.41887438, ..., 0.        , 0.        ,
        0.        ],
       [0.29593462, 0.34047148, 0.56619775, ..., 0.        , 0.        ,
        0.        ],
       [0.27839258, 0.32688344, 0.4162595 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>>

In [27]:
generated_images = generator(tf.random.normal([1000, noise_dim]))


In [28]:
generated_images

<tf.Tensor: shape=(1000, 147), dtype=float32, numpy=
array([[0.41432533, 0.65544325, 0.28528917, ..., 0.        , 0.        ,
        0.        ],
       [0.3588624 , 0.42854884, 0.41608822, ..., 0.        , 0.        ,
        0.        ],
       [0.53015906, 0.08060289, 0.5188161 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.6150852 , 0.5877808 , 0.5859526 , ..., 0.        , 0.        ,
        0.        ],
       [0.5114996 , 0.55857164, 0.22718066, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03506737, 0.2801032 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [29]:
col = np.load('gitCol.npy', allow_pickle=True)

In [30]:
col.shape

(147,)

In [31]:
generated = pd.DataFrame(generated_images, columns=col)

In [32]:
save = generated.to_pickle("gen_data3.pkl")


In [ ]:
generated

In [ ]:

plt.figure(figsize=(10,8))
plt.plot(list(range(0,EPOCHS)),gloss)
plt.plot(list(range(0,EPOCHS)),dloss)
plt.title('Model Loss over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['generator loss','discriminator loss'])
plt.savefig('Learning_curve gitdnn')
plt.show
plt.close()